In [ ]:
!pip install pyyaml easydict munkres

In [ ]:
%pylab notebook
import os
import sys
import cv2
from imageio import imread
from random import randint
import numpy as np
import tensorflow as tf
from config import load_config
from nnet.net factory import pose_net

In [ ]:
def setup_pose_prediction(cfg):
    inputs = tf.placeholder(tf.float32, shape=[cfg.batch_size, None, None, 3])

    outputs = pose_net(cfg).test(inputs)

    restorer = tf.train.Saver()

    sess = tf.Session()

    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())

    # Restore variables from disk.
    restorer.restore(sess, cfg.init_weights)

    return sess, inputs, outputs

In [ ]:
def extract_cnn_output(outputs_np, cfg, pairwise_stats = None):
    scmap = outputs_np['part_prob']
    scmap = np.squeeze(scmap)
    locref = None
    pairwise_diff = None
    if cfg.location_refinement:
        locref = np.squeeze(outputs_np['locref'])
        shape = locref.shape
        locref = np.reshape(locref, (shape[0], shape[1], -1, 2))
        locref *= cfg.locref_stdev
    if cfg.pairwise_predict:
        pairwise_diff = np.squeeze(outputs_np['pairwise_pred'])
        shape = pairwise_diff.shape
        pairwise_diff = np.reshape(pairwise_diff, (shape[0], shape[1], -1, 2))
        num_joints = cfg.num_joints
        for pair in pairwise_stats:
            pair_id = (num_joints - 1) * pair[0] + pair[1] - int(pair[0] < pair[1])
            pairwise_diff[:, :, pair_id, 0] *= pairwise_stats[pair]["std"][0]
            pairwise_diff[:, :, pair_id, 0] += pairwise_stats[pair]["mean"][0]
            pairwise_diff[:, :, pair_id, 1] *= pairwise_stats[pair]["std"][1]
            pairwise_diff[:, :, pair_id, 1] += pairwise_stats[pair]["mean"][1]
    return scmap, locref, pairwise_diff

In [ ]:
def argmax_pose_predict(scmap, offmat, stride):
    """Combine scoremat and offsets to the final pose."""
    num_joints = scmap.shape[2]
    pose = []
    for joint_idx in range(num_joints):
        maxloc = np.unravel_index(np.argmax(scmap[:, :, joint_idx]),
                                  scmap[:, :, joint_idx].shape)
        offset = np.array(offmat[maxloc][joint_idx])[::-1] if offmat is not None else 0
        pos_f8 = (np.array(maxloc).astype('float') * stride + 0.5 * stride +
                  offset)
        pose.append(np.hstack((pos_f8[::-1],
                               [scmap[maxloc][joint_idx]])))
    return np.array(pose)

In [ ]:
cfg = load_config("demo/pose_cfg.yaml")
sess, inputs, outputs = setup_pose_prediction(cfg)

In [ ]:
file_name = "testcases/standing-lef-lift.jpg"
image = np.array(imread(file_name))
image_batch = np.expand_dims(image, axis=0).astype(float)
outputs_np = sess.run(outputs, feed_dict={inputs: image_batch})
scmap, locref, pairwise_diff = extract_cnn_output(outputs_np, cfg)
pose = argmax_pose_predict(scmap, locref, cfg.stride)

In [ ]:
pose2D = pose[:, :2]
image_annot = image.copy()

for index in range(5):
    randcolor = tuple([randint(0, 255) for i in range(3)])
    thickness = int(min(image_annot[:,:,0].shape)/250) + 1
    start_pt = tuple(pose2D[index].astype('int'))
    end_pt = tuple(pose2D[index+1].astype('int'))
    image_annot = cv2.line(image_annot, start_pt, end_pt, randcolor, thickness)
for index in range(6,11): #next bunch are arms/shoulders (from one hand to other)
    randcolor = tuple([randint(0,255) for i in range(3)])
    thickness = int(min(image_annot[:,:,0].shape)/250) + 1
    start_pt = tuple(pose2D[index].astype('int'))
    end_pt = tuple(pose2D[index+1].astype('int'))
    image_annot = cv2.line(image_annot, start_pt, end_pt, randcolor, thickness)
#connect Line from chin to top of head
image_annot = cv2.line(image_annot,
                       tuple(pose2D[12].astype('int')), tuple(pose2D[13].astype('i
                       tuple([randint(0,255) for i in range(3)]), thickness)

In [ ]:
figure()
imshow(image)

In [ ]:
# There no actual joints on waist or coLLar,
# but we can estimate them from hip/shoulder midpoints
waist = tuple(((pose2D[2]+pose2D[3])/2).astype('int'))
collar = tuple(((pose2D[8]+pose2D[9])/2).astype('int'))
# draw the "spine"
image_annot = cv2.line(image_annot, waist, collar,
                       tuple([randint(0,255) for i in range(3)]), thickness)
image_annot = cv2.line(image_annot, tuple(pose2D[12].astype('int')), collar,
                       tuple([randint(0,255) for i in range(3)]), thickness)
# now Label the joints with numbers
font = cv2.FONT_HERSHEY_SIMPLEX
fontsize = min(image_annot[:,:,0].shape)/750 #scale the font size to the image size
for idx, pt in enumerate(pose2D):
    randcolor = tuple([randint(0,255) for i in range(3)])
image_annot = cv2.putText(image_annot, str(idx+1),
                          tup1e(pt.astype('int')),font, fontsize,
                          randcolor,2,cv2.LINE_AA)
figure()
imshow(image_annot)

In [ ]:
import os
import sys
import numpy as np
import cv2 I
from imageio import imread, imsave
from config import load_config
from dataset.factory import create as create_dataset
from nnet import predict
from dataset.pose_dataset import data_to_input
from multiperson.detections import extract_detections
from multiperson.predict import SpatialModel, eval_graph, get_person_conf_mu1ticut
# from muLtiperson.visuaLize import PersonDraw, visuaLize_detections

In [ ]:
cf = load_config("demo/pose_cfg_multi.yaml) 

In [ ]:
dataset = create_dataset(cfg)
sm = SpatialModel(cfg)
sm.load()
sess, inputs, outputs = predict.setup_pose_prediction(cfg)

In [ ]:
file_name = "testcases/bus_people.jpg"
image = np.array(imread(file_name))
image_batch = data_to_input(image)
# Compute prediction with the CNN
outputs_np = sess.run(outputs, feed_dict={inputs: image_batch})
scmap, locref, pairwise_diff = predict.extract_cnn_output(outputs_np, cfg, dataset
detections = extract_detections(cfg, scmap, locref, pairwise_diff)
unLab, pos_array, unary_array, pwidx_array, pw_array = eval_graph(sm, detections)
person_conf_multi = get_person_conf_multicut(sm, unLab, unary_array, pos_array)
image_annot = image.copy()
for pose2D in person_conf_mu1ti:
    font = cv2.FONT_HERSHEY_SIMPLEX
    fontsize = min(image_annot[:,:,0].shape)/1000

In [ ]:
predictor_path = "./shape_predictor_68_face_landmarks.dat"
detector = dlib.get_fronta1_face_detector()
predictor = dlib.shape_predictor(predictor_path)

#Uncomment Line below if you want to use your webcam
#cap = cv2.VideoCapture(0) #0 is the first camera on your computer, change if you
#more than one camera

#Comment out the Line below if using webcam
cap = cv2.VideoCapture('./rollerc.mp4')
figure(100)
font = cv2.FONT_HERSHEY_SIMPLEX

In [ ]:
font = cv2.FONT_HERSHEY_SIMPLEX
while(True):
    #Capture frame-by-frame
    ret, img = cap.read()
    img.flags['WRITEABLE']=True #just in case
  
    try:
        dets = detector(img, 1)
        shape = predictor(img, dets[0])
    except:
        print('no face detected', end='\r')
        cap.release()
        break
#similar to previous example, except frame-by-frame here
    annotated=img.copy()
    head_width = shape.part(16).x-shape.part(6).x
    fontsize = head_width/650
    for pt in range(68):
        x,y = shape.part(pt).x, shape.part(pt).y
        annotated=cv2.putText(annotated, str(pt), (x,y), font, fontsize, (255,255,255), 2, cv2.LINE_AA)

#Let’s see our resuLts
    fig=imshow(cv2.cvtColor(annotated,cv2.COLOR_BGR2RGB)) #OpenCV uses BGR format

    display.c1ear_output(wait=True)
    display.display(gcf())

#When everything is done, release the capture
cap.release()